In [1]:
import sys
import logging
sys.path.append('../')
import torch
from torch.utils.data import DataLoader
from src.data.loaders import *
from src.data.utils import *

In [2]:
# Setup basic logger
logging.basicConfig(level=logging.DEBUG)
log = logging.getLogger(__name__)

In [3]:
mydevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(mydevice)
update_collate(mydevice)

cuda:0


In [4]:
# Try to set up a data loader and see if it works
print('Creating Preprocessor')
preprocessor = Preprocessor(device=mydevice)
print('Creating Data Store')
data_store = DataStore(
    data_dir='C:/LOFAR/subsubset/',
    preprocessor=preprocessor,
    device=mydevice,
    uv=True
)
print('Creating Dataset')
dataset = LOFARDataset(
    data=data_store,
    preprocessor=preprocessor,
    uv=True
)
log.debug(f"Dataset length: {len(dataset)}")
log.debug(f"Dataset map: {data_store.map}")
print('Creating Data Loader')
data_loader = DataLoader(
    dataset=dataset,
    batch_size=1,
    shuffle=True,
    collate_fn=collate_fn['patch'],
)
print('Done.')

INFO:src.data.loaders:Creating data store on device: cuda:0
DEBUG:src.data.utils:[get_fileSAP] Processing file C:/LOFAR/subsubset\L701795.MS_extract.h5.
DEBUG:src.data.utils:Removed 0 duplicate mapping entries.
INFO:src.data.loaders:Mapped dataset in C:/LOFAR/subsubset/. 2850 baselines.
DEBUG:src.data.loaders:Getting observation 632577 and SAP 0


Creating Preprocessor
Creating Data Store


DEBUG:h5py._conv:Creating converter from 3 to 5
DEBUG:__main__:Dataset length: 2850
DEBUG:__main__:Dataset map: {632577: 'C:/LOFAR/subsubset\\L701795.MS_extract.h5'}


Creating Dataset
Creating Data Loader
Done.


In [5]:
# Iterate through the data loader and see if it works
counter = 0
for x,_ in data_loader:
    counter += 1
    print('iter')
    log.info(x.shape)
    if counter == 10:
        break

INFO:src.data.loaders:Getting baseline torch.Size([2, 4, 128, 128]) with uv torch.Size([2, 2]).
INFO:__main__:torch.Size([1, 256, 128, 128])
INFO:src.data.loaders:Getting baseline torch.Size([2, 4, 128, 128]) with uv torch.Size([2, 2]).
INFO:__main__:torch.Size([1, 256, 128, 128])
INFO:src.data.loaders:Getting baseline torch.Size([2, 4, 128, 128]) with uv torch.Size([2, 2]).
INFO:__main__:torch.Size([1, 256, 128, 128])
INFO:src.data.loaders:Getting baseline torch.Size([2, 4, 128, 128]) with uv torch.Size([2, 2]).
INFO:__main__:torch.Size([1, 256, 128, 128])
INFO:src.data.loaders:Getting baseline torch.Size([2, 4, 128, 128]) with uv torch.Size([2, 2]).
INFO:__main__:torch.Size([1, 256, 128, 128])
INFO:src.data.loaders:Getting baseline torch.Size([2, 4, 128, 128]) with uv torch.Size([2, 2]).
INFO:__main__:torch.Size([1, 256, 128, 128])
INFO:src.data.loaders:Getting baseline torch.Size([2, 4, 128, 128]) with uv torch.Size([2, 2]).
INFO:__main__:torch.Size([1, 256, 128, 128])
INFO:src.data

iter
iter
iter
iter
iter
iter
iter
iter
iter
iter


In [6]:
from hydra.experimental import compose, initialize
from hydra.utils import instantiate
initialize(config_path="../config")
cfg = compose(config_name="config")

c:\Users\Klemen\mambaforge\envs\msproj\Lib\site-packages\hydra\experimental\initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
c:\Users\Klemen\mambaforge\envs\msproj\Lib\site-packages\hydra\experimental\initialize.py:45: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  self.delegate = real_initialize(
DEBUG:hydra.core.utils:Setting JobRuntime:name=UNKNOWN_NAME
DEBUG:hydra.core.utils:Setting JobRuntime:name=notebook
c:\Users\Klemen\mambaforge\envs\msproj\Lib\site-packages\hydra\experimental\compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(message=message)


In [7]:
lofar_data_module = LOFARDataModule(
    data_conf=cfg.data,
    preproc_conf=cfg.preproc,
    loader_conf=cfg.loader,
)

INFO:src.data.loaders:Creating data store on device: cuda:0
DEBUG:src.data.utils:[get_fileSAP] Processing file C:/LOFAR/subsubset\L701795.MS_extract.h5.
DEBUG:src.data.utils:Removed 0 duplicate mapping entries.
INFO:src.data.loaders:Mapped dataset in C:/LOFAR/subsubset/. 2850 baselines.
DEBUG:src.data.loaders:Getting observation 632577 and SAP 0


In [8]:
model = instantiate(cfg.model)

BasicBottleneck: in_units=131072, out_units=131072, latent=256
